In [1]:
# import dependencies
import requests as req
import json
import pymongo

In [6]:
# get JSON from API
# TODO: choose a different game
data = req.get('https://statsapi.web.nhl.com/api/v1/game/2017030224/feed/live').json()

In [7]:
# grab all plays from game
plays = data['liveData']['plays']['allPlays']

In [46]:
# get player names involved in play
for play in plays:
    event = play['result']['event']
    
    if event == 'Shot':
        shooter = play['players'][0]['player']['fullName']
        goalie = play['players'][1]['player']['fullName']
    elif event == 'Goal':
        for player in play['players']:  
            print(player['player']['fullName'])
        shooter = play['players'][0]['player']['fullName']
        goalie = play['players'][1]['player']['fullName']
    elif event == 'Hit':
        hitter = play['players'][0]['player']['fullName']

Nathan Beaulieu
Brendan Gallagher
Alex Galchenyuk
Ryan Miller
Torrey Mitchell
Phillip Danault
Brian Flynn
Ryan Miller
Alexander Radulov
Alex Galchenyuk
Brendan Gallagher


In [47]:
# create lists to sort into
shots = []
goals = []
hits = []

In [48]:
# sort events into lists
for play in plays:
    
    event = play['result']['event']
    
    if event == 'Shot':
        d = {
        'event' : '',
        'shooter' : '',
        'goalie' : '',
        'x' : '',
        'y' : ''
            }
        shooter = play['players'][0]['player']['fullName']
        goalie = play['players'][1]['player']['fullName']
        x = play['coordinates']['x']
        y = play['coordinates']['y']
        d['event'] = event
        d['shooter'] = shooter
        d['goalie'] = goalie
        d['x'] = x
        d['y'] = y
    elif event == 'Goal':
        d = {
        'event' : '',
        'shooter' : '',
        'goalie' : '',
        'x' : '',
        'y' : ''
                    }
        for player in play['players']:  
            if player['playerType'] == 'Scorer':      
                shooter = player['player']['fullName']
            elif player['playerType'] == 'Goalie':
                goalie = player['player']['fullName']
        x = play['coordinates']['x']
        y = play['coordinates']['y']
        d['event'] = event
        d['shooter'] = shooter
        d['goalie'] = goalie
        d['x'] = x
        d['y'] = y
    elif event == 'Hit':
        d = {
        'event' : '',
        'hitter' : '',
        'x' : '',
        'y' : ''
        }
        hitter = play['players'][0]['player']['fullName']
        x = play['coordinates']['x']
        y = play['coordinates']['y']
        d['event'] = event
        d['hitter'] = hitter
        d['x'] = x
        d['y'] = y
    
    
    if event == 'Shot':
        shots.append(d)
    elif event == 'Goal':
        goals.append(d)
    elif event == 'Hit':
        hits.append(d)

In [49]:
goals

[{'event': 'Goal',
  'shooter': 'Nathan Beaulieu',
  'goalie': 'Ryan Miller',
  'x': 82.0,
  'y': 0.0},
 {'event': 'Goal',
  'shooter': 'Torrey Mitchell',
  'goalie': 'Ryan Miller',
  'x': 79.0,
  'y': -4.0},
 {'event': 'Goal',
  'shooter': 'Alexander Radulov',
  'goalie': 'Carey Price',
  'x': -77.0,
  'y': 3.0}]

In [50]:
# connect to db
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [51]:
db = client.nhlDB

In [52]:
# add shot coordinates to db
# TODO: add more info (player, team, period, etc.)
db.shots.insert_many(shots)

In [53]:
# add goal coordinates to db
# TODO: add more info (player, team, period, etc.)
db.goals.insert_many(goals)

In [54]:
# add hit coordinates to db
# TODO: add more info (player, team, period, etc.)
db.hits.insert_many(hits)